<a href="https://colab.research.google.com/github/jetsonmom/cnn/blob/main/lane_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from google.colab.patches import cv2_imshow

import cv2
import numpy as np
import math
print(tf.__version__)
print(cv2.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# video_path = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/1.mp4'
video_path = '/content/drive/MyDrive/1.mp4'

video_capture = cv2.VideoCapture(video_path)

PERSPECTIVE_IMG_W = 640
PERSPECTIVE_IMG_H = 480
BASE_LINE  = 340
BASE_LINE_ROI_WIDTH = 20

save_file_path = './images/original/'
google_save_file_path = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/images/'
save_file_path_crop = './images/crop/'
google_save_file_path_crop = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/images/crop/'

In [ ]:
import os
import shutil

def DeleteAllFiles(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
    else:
        os.makedirs(filePath)
        return 'Directory not Found and make director'

filePath1 = './images/original'
print(DeleteAllFiles(filePath1))


filePath2 = './images/crop'
print(DeleteAllFiles(filePath2))

filePath3 = './images/lane_data'
print(DeleteAllFiles(filePath3))

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

영상에서 이미지 만드는 코드

In [ ]:
import cv2
import os

video_path = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/1.mp4'
cap = cv2.VideoCapture(video_path)

save_dir = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/extracted_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break  # 더 이상 읽을 프레임이 없으면 종료

    # 10 프레임마다 이미지 저장
    if frame_count % 10 == 0:
        cv2.imwrite(os.path.join(save_dir, f'frame_{frame_count}.jpg'), frame)
        print(f'Saved frame number: {frame_count}')  # 저장된 프레임 번호 출력 (진행 상황 확인용)

    frame_count += 1

# 모든 작업 완료 후 비디오 파일 닫기
cap.release()


In [ ]:
test_image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/images/frame_0.jpg')
height = test_image.shape[0]
width  = test_image.shape[1]

print(height,width)
cv2_imshow(test_image)

## 영상처리 알고리즘 함수

In [ ]:
def canny_edge_detector(image):

    # Convert the image color to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Reduce noise from the image
    blur = cv2.GaussianBlur(gray_image, (5, 5), 0)
    canny = cv2.Canny(blur, 40, 130)
    return canny


def extract_image_region_of_interest(image):
    height = image.shape[0]
    width  = image.shape[1]
    crop_image = image[ BASE_LINE  - BASE_LINE_ROI_WIDTH : BASE_LINE  + BASE_LINE_ROI_WIDTH, 0: width]

    return crop_image


def OpenCV_Perspective_Calibration(image):

    Source = np.float32([[0,270],[-600,480],[640,270],[640+600,480]])

    Destination = np.float32([[0,0],[0,PERSPECTIVE_IMG_H], [PERSPECTIVE_IMG_W,0],[PERSPECTIVE_IMG_W,PERSPECTIVE_IMG_H]])
    Matrix = cv2.getPerspectiveTransform(Source, Destination)
    perspective_image = cv2.warpPerspective(image, Matrix, (PERSPECTIVE_IMG_W,PERSPECTIVE_IMG_H))

    return perspective_image


def average_slope_intercept(lines):
    left_fit = []
    right_fit = []
    center_fit = []
    slope_data =[]

    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        #print x1, y1, x2, y2
        #cv2.line(image,(x1,y1),(x2,y2),(0,255,255),1)
        if y1 != y2 :
            parameters = np.polyfit((y1, y2), (x1, x2), 1)
            slope = parameters[0]
            intercept = parameters[1]
            slope_data.append(slope)

    slope_avg = sum(slope_data) / len(slope_data)

    slope_avg_degree = -math.atan(slope_avg)*180/3.14159

    print('lane slope = %6.2lf'%(slope_avg_degree))

    return slope_avg_degree


def lane_image(image, lines):
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        cv2.line(image,(x1,y1),(x2,y2),255,2)
    return image

In [ ]:
test_image_crop = extract_image_region_of_interest(test_image)

height_crop = test_image_crop.shape[0]
width_crop  = test_image_crop.shape[1]
cv2_imshow(test_image_crop)

In [ ]:
video_capture = cv2.VideoCapture(video_path)

count=0

if video_capture.isOpened() == False:
    print("video file does not exit")

while(video_capture.isOpened()):

    success, image =video_capture.read()   # 캡쳐된 이미지를 저장하는 함수

    if success:
        num_str = str(count).zfill(4)
        #save_file_name = google_save_file_path + num_str + '.jpg'
        save_file_name = save_file_path + num_str + '.jpg'
        #save_file_crop_name = google_save_file_path_crop + 'crop_' + num_str + '.jpg'
        save_file_crop_name = save_file_path_crop + 'crop_' + num_str + '.jpg'
        print(save_file_name)
        image_crop = extract_image_region_of_interest(image)

        cv2.imwrite(save_file_name,image)
        cv2.imwrite(save_file_crop_name,image_crop)

        count +=1

    else :
       print("No stream")
       break
video_capture.release()


print("finish! convert video to frame")

In [ ]:
test_image_crop = extract_image_region_of_interest(test_image)
edge_image = canny_edge_detector(test_image_crop)
cv2_imshow(edge_image)
print()
kernel = np.ones((7,7), np.uint8)
cv_dilataion = cv2.dilate(edge_image, kernel, 3)

cv2_imshow(cv_dilataion)

In [ ]:
import os

crop_image_dir = 'images/crop'
crop_img_size = (480,640)


input_crop_img_paths = sorted([os.path.join(crop_image_dir, fname)
                         for fname in os.listdir(crop_image_dir)
                         if fname.endswith('.jpg') ])

no_file  = len(input_crop_img_paths)
print("crop image :",no_file)

#input_img_paths[1]

image_dir = 'images/original'
img_size = (480,640)


input_img_paths = sorted([os.path.join(image_dir, fname)
                         for fname in os.listdir(image_dir)
                         if fname.endswith('.jpg') ])

no_file  = len(input_img_paths)
print("original image :",no_file)


In [ ]:
cv_line_image_empty = np.ones((height_crop,width_crop), dtype=np.uint8)
    #cv2_imshow(cv_line_image_empty)
cv_line_image = cv_line_image_empty.copy()
    #cv2_imshow(cv_line_image)

def line_detect(image):
    roi_image = extract_image_region_of_interest(image)

    cv_edge_roi_image = canny_edge_detector(roi_image)
    kernel = np.ones((9,9), np.uint8)
    cv_dilataion = cv2.dilate(cv_edge_roi_image, kernel, 3)

    #cv2_imshow(cv_dilataion)

    minLineLength = 10
    maxLineGap = 5

    cv_edge_roi_image_color = cv2.cvtColor(cv_edge_roi_image, cv2.COLOR_GRAY2RGB)
    lines = cv2.HoughLinesP(cv_edge_roi_image, 2, np.pi / 180, 20,np.array([]), minLineLength,maxLineGap)
    cv_line_image = lane_image(cv_line_image_empty, lines)
    avg_line_slope = average_slope_intercept(lines)
    #print(lines)



    return cv_dilataion ,cv_line_image, avg_line_slope

In [ ]:
def cal_line_center(image,angle):



    label_no, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    line_center_pos =  np.zeros(label_no-1)
    line_center = 0

#angle = average_slope_intercept(cv_edge_roi_image_color, lines)
    cv_line_image_color = cv2.cvtColor(cv_line_image, cv2.COLOR_GRAY2RGB)
    for i in range(1, label_no): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        (center_x,center_y) = centroids[i]
        line_center_pos[i-1] = center_x
        #print(x,y,w,h,area)
        #print(center_x,center_y)
        if area < 50:
            continue
        #cv2.rectangle(test_image_crop, (x, y, w, h), (0, 255, 255))
        cv2.rectangle(test_image_crop, (int(center_x)-5,0, 10, 40), (0, 255, 0))



    line_center_pos.sort()

    if label_no == 1:
        print('no_lane')
    elif label_no == 2:
        print('1 lane')
        if(angle<0):
            line_center = line_center_pos[0] - 306
            cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))
        elif(angle>0):
            line_center = line_center_pos[0] + 259
            cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))
        cv2_imshow(test_image_crop)

    elif label_no == 3:
        print('2 lane')
        lane_width  = line_center_pos[1]-line_center_pos[0]
        print('lane width :', lane_width)

        if (lane_width >=400):
            if(angle<0):
                line_center = line_center_pos[1] - 306
                cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))
            elif(angle>0):
                line_center = line_center_pos[0] + 259
                cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))


        elif (lane_width <400) & (lane_width >=230):
            if(angle<0):
                line_center = line_center_pos[0]
                cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))
            elif(angle>0):
                line_center = line_center_pos[1]
                cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))



        elif (lane_width < 230):
            line_center = line_center_pos[1]
            cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))


        cv2_imshow(test_image_crop)
    elif label_no == 4:
        print('3 lanes')
        line_center = line_center_pos[1]
        cv2.line(test_image_crop, (int(line_center),0), (int(line_center),BASE_LINE_ROI_WIDTH*2),(255, 0, 0))

        cv2_imshow(test_image_crop)
    else :
        print('too many lane')

    return line_center

코드 설명
i = 100:
i 변수에 100이라는 값을 할당합니다. 이는 일반적으로 인덱스나 순번을 나타내며, 배열이나 리스트에서 특정 위치의 요소를 참조할 때 사용됩니다.
print(input_img_paths[i]):
input_img_paths 리스트에서 인덱스 100에 해당하는 요소의 값을 출력합니다. 이 값은 이미지 파일의 경로일 가능성이 높습니다.
test_image = cv2.imread(input_img_paths[i]):
OpenCV의 cv2.imread() 함수를 사용하여 input_img_paths 리스트의 100번째 요소(이미지 파일 경로)를 읽어들여 test_image 변수에 저장합니다.
test_image_crop = cv2.imread(input_crop_img_paths[i]):
input_crop_img_paths 리스트의 100번째 요소를 사용하여 특정 부분이 자른(cropped) 이미지를 읽어들입니다.
lane_dilation_img, cv_line_image, line_angle = line_detect(test_image):
line_detect 함수를 호출하여 test_image에서 차선을 감지합니다. 이 함수는 세 개의 결과를 반환합니다:
lane_dilation_img: 차선 감지 후 처리된 이미지(확대된 차선 등을 포함할 수 있음).
cv_line_image: 차선이 표시된 이미지.
line_angle: 감지된 차선의 각도.
line_center = cal_line_center(lane_dilation_img, line_angle):
cal_line_center 함수를 사용하여 lane_dilation_img 이미지와 line_angle을 바탕으로 차선의 중심을 계산합니다.
결과 메시지 출력:
"images/original/0100.jpg"는 처리된 이미지의 경로를 나타냅니다.
"lane slope = 35.01"는 감지된 차선의 기울기를 나타냅니다.
"too many lane"은 감지된 차선이 너무 많음을 나타내는 메시지입니다.

In [ ]:
i = 100
print(input_img_paths[i])
test_image = cv2.imread(input_img_paths[i])
test_image_crop = cv2.imread(input_crop_img_paths[i])
lane_dilation_img, cv_line_image, line_angle  = line_detect(test_image)
line_center = cal_line_center(lane_dilation_img, line_angle)

In [ ]:
#for i in range(len(input_img_paths)):

result_save_file_path = './images/lane_data/'
google_result_save_file_path = '/content/drive/MyDrive/Colab Notebooks/Image Processing/Lane Detection/images/lane_center_data/'

for i in range(1400):
    print(input_img_paths[i])

    num_str = str(i).zfill(4)
    save_file_name = google_result_save_file_path + 'lane_center_'+ num_str + '.txt'
    #print(save_file_name)
    line_center = 10.1
    f = open(save_file_name,'w')
    test_image = cv2.imread(input_img_paths[i])
    test_image_crop = cv2.imread(input_crop_img_paths[i])
    lane_dilation_img, cv_line_image, line_angle  = line_detect(test_image)
    line_center = cal_line_center(lane_dilation_img, line_angle)
    data = '%4.1f'%line_center
    f.write(data)
    f.close

    print('\n')

학습 데이터 중에서 1400까지만 할 것

```
# 코드로 형식 지정됨
```



In [ ]:
for j in range(10):
    num_str1 = str(j).zfill(4)

    #num_str1.rjust(4,'0')
    print(num_str1)